In [1]:
# Prediction of Health Measures Based on Location

**Data Scientist:** Elina Rankova

<p align="center">
  <img src="https://media.licdn.com/dms/image/C4D12AQE3ln6Z4Mo8Pw/article-cover_image-shrink_720_1280/0/1642087174753?e=1718236800&v=beta&t=etvT9WV2qS3g7hAnCds-vQIn3ob9JqJn9vvB9annTWM">
</p>
<a href="https://www.linkedin.com/pulse/benefits-population-health-management-phm-amazing-charts/">Image Source</a>

SyntaxError: invalid syntax (4139135345.py, line 3)

## 1. Business Problem and Understanding

**Stakeholders:** Company executives specializing in operational expansion to improve public health prevention where it is most needed.

As technology evolves, public health and healthcare in particular has long been been lagging behind. In recent years, more companies are leaning on data and technology to inform growth opportunities with the goal of helping those most in need.

As an independent contractor specializing in expanding resource availability to undeserved populations, this project aims to identify overall health disparity across the United States utilizing public county and census data provided by the CDC. Proving population health measure specifics based on an overall Health Disparity Index can help direct those striving to close care gaps to expand where there is the greatest need.

For Phase 1 of this initiative we will define a Health Disparity Index (HDI) based on a number of health measures and population by geolocation.

**The goal:** Provide clients with a holistic understanding of the measures contributing to an overall Health Disparity Index aggregated by geolocation to better understand what United States regions most provide opportunity for growth.

In [ ]:
# Import libraries
import pandas as pd
import geopandas as gpd
from shapely import wkt
import numpy as np
import random
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import zscore
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedShuffleSplit, cross_validate
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, \
GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.base import clone
from sklearn.base import BaseEstimator, RegressorMixin
from metrics_cv import MetricsCV
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.regularizers import l1, l2, l1_l2
# from scikeras.wrappers import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
MetricsCV??

## 2. Data Understanding

For this task we will be using CDC provided PLACES and SDOH county data spanning years 2017-2021
Datasets
- <a href="https://data.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-County-Data-20/dv4u-3x3q/about_data">Local Data for Better Health, County Data 2020 release</a>
- <a href="https://data.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-County-Data-20/pqpp-u99h/about_data">Local Data for Better Health, County Data 2021 release</a>
- <a href="https://data.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-County-Data-20/duw2-7jbt/about_data">Local Data for Better Health, County Data 2022 release</a>
- <a href="https://data.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-County-Data-20/swc5-untb/about_data">Local Data for Better Health, County Data 2023 release</a>
- <a href="https://data.cdc.gov/500-Cities-Places/SDOH-Measures-for-County-ACS-2017-2021/i6u4-y3g4/about_data">SDOH Measures for County, ACS 2017-2021</a>

Utilizing all the health measures defined within the cagtegories specified below we will create a custom Health Disparity Index (HDI) as our target for this regression task.

**Categories within which health measures are defined:** Health Outcomes, Prevention, Health Risk Behaviors, Health Status, Disabilities, and Social Determinants of Health

In [ ]:
# Load datasets

pl1 = pd.read_csv('Data/PLACES_2017-2018.csv')
pl2 = pd.read_csv('Data/PLACES_2018-2019.csv')
pl3 = pd.read_csv('Data/PLACES_2019-2020.csv')
pl4 = pd.read_csv('Data/PLACES_2020-2021.csv')
sdoh = pd.read_csv('Data/SDOH_2017-2021.csv')

We may want to check on the `StateAbbr` and `StateDesc` since it's showing the country in the preview. We also want to adjust the LocationName to either exclude 'county' or include 'county' to make sure both of the datasets are aligned in nomenclature.

Since the data we are working with is by county, we can eliminate the added specification with the `LocationName` in the SDOH dataset.

We will also have to deal with the differences in the `Year` column since it is an object and range in the SDOH data and an integer in the PLACES data.

In [ ]:
# Concat all PLACES datasets
pl_all = pd.concat([pl1, pl2, pl3, pl4])
display(pl_all.tail())

sdoh.head()

In [ ]:
# Lowercase all data to ensure consistency
sdoh = sdoh.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
pl_all = pl_all.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

# Delete the 'county' specification
sdoh['LocationName'] = sdoh['LocationName'].str.replace('county',"")

In [ ]:
print(sdoh.info())
print(pl_all.info())

We will have to drop `Low_Confidence_Limit` and `High_Confidence_Limit` since the SDOH dataset does not have these columns.

In [ ]:
print(sdoh.isna().sum()/sdoh.shape[0],'\n')
print(pl_all.isna().sum()/pl_all.shape[0])

In [ ]:
# Merge all datasets together
df_all = pd.concat([pl_all, sdoh])

Right away we can tell that there are a few columns that are missing too many values and will need to be dropped.
- `Data_Value_Footnote_Symbol`
- `Data_Value_Footnote`
- `Latitude`
- `MOE`

`Geolocatioin` and `Geolocation` are the same feature when checking the source data websites only the 2020 release has a `Geolocatioin` column while the rest have `Geolocation`. `MeasureID` and `MeasureId` have the same issue it seems.

There are object type columns that will need to be transformed. Since state should be treated as a categorical variable we should adjust this in our dataset.

In addition, we would also want to make `LocationID` an object since this is not an actual continuous variable.

In [ ]:
print(df_all.info(),'\n')

# Drop unneeded columns
df_all.drop(columns=['Data_Value_Footnote_Symbol','Data_Value_Footnote','Latitude',
                     'MOE','Low_Confidence_Limit','High_Confidence_Limit'],inplace=True)

We can drop the misspelled `Geolocatioin` after filling in the nas in `Geolocation`. This concept applies to `MeasureID` and `MeasureId` as well.

In [ ]:
# Fill NaNs in Geolocation and with Geolocatioin
df_all.loc[df_all['Geolocation'].isna(), 'Geolocation'] = df_all.loc[df_all['Geolocation'].isna(), 'Geolocatioin']

# Apply the same method to `MeasureID
df_all.loc[df_all['MeasureID'].isna(), 'MeasureID'] = df_all.loc[df_all['MeasureID'].isna(), 'MeasureId']

# Drop unneeded columns
df_all.drop(columns=['MeasureId','Geolocatioin'],axis=1,inplace=True)

df_all.info()

We can drop nas in any columns we are not already dropping such as `LocationName` and the updated `Geolocation`. `LocationID` refers to the zipcode so we should first see if we can impute with the proper zipcode before droping nas.

In [ ]:
print(df_all.isna().sum()/df_all.shape[0])

df_all.dropna(subset=['LocationName','Geolocation'],inplace=True)

Looks like we were able to fill the nas using the associated `LocationID` with existing `Geolocation` as a unique identifier for the zipcode. 

In [ ]:
# Filter rows with non-null LocationID
loc_df = df_all.loc[df_all['LocationID'].notna(), ['Geolocation', 'LocationID']]

# Create LocationName <> LocationID dictionary
loc_dict = dict(zip(loc_df['Geolocation'], loc_df['LocationID']))

# Fill missing LocationID values based on LocationName from the dictionary
df_all['LocationID'] = df_all['LocationID'].fillna(df_all['Geolocation'].map(loc_dict))

# Drop the rest of the nas in LocationID
df_all.dropna(inplace=True)

# Check the percentage of missing values in df_all after filling missing values
print(df_all.isna().sum() / df_all.shape[0])

In [ ]:
# Convert 'StateAbbr' and 'LocationID' columns to appropriate type
df_all['StateAbbr'] = df_all['StateAbbr'].astype('category')
df_all['LocationID'] = df_all['LocationID'].astype('object')

### Feature Exploration

<ins>**Observations**</ins>

- We will likely drop `Year` as it can only provide relevant information for the data we retrieved from the PLACES datasets. Otherwise, we have the same date range as seen in the added SDOH data (2017-2021).
- Looks like we may want to drop the rows with US as the state as it is an error. We can also drop `StateDesc` since it's the same info as `StateAbbr`.
- `DataSource` can be dropped since all of the information is coming from the Behavioral Risk Factor Surveillance System or the 5-year American Community Survey which only contains SDOH data.
- `Data_Value_Unit` can be dropped since all of our data values are in percentages.
- `Data_Value_Type` will be helpful for interpretation of our predictions since it denotes what percentage the `Data_Value` represents. `DataValueTypeID` corresponds to this feature and can be dropped since `Data_Value_Type` is more informative. 
- `CategoryID` corresponds to `Category` so we can drop it since `Category` is easier to interpret.
- `MeasureId` corresponds the same way to `Measure`. However `Measure` values can be quite lengthy depending on the measure. We also have `Short_Question_Text` corresponding to these features and is more informative than `MeausureId` but shorter than `Measure` so we will keep `Short_Question_Text` and create a reference dictionary before dropping the other columns.

In [ ]:
print(df_all['Year'].value_counts(),'\n')
print(df_all['StateAbbr'].value_counts(),'\n')
print(df_all['DataSource'].value_counts(),'\n')
print(df_all['Data_Value_Unit'].value_counts(),'\n')
print(df_all['Data_Value_Type'].value_counts(),'\n')
print(df_all['CategoryID'].value_counts(),'\n')
print(df_all['MeasureID'].value_counts(),'\n')
print(df_all['DataValueTypeID'].value_counts(),'\n')
print(df_all['Short_Question_Text'].value_counts(),'\n')

In [ ]:
# Create reference dictionary for category and measure
measure_df = df_all[['Measure','Short_Question_Text','MeasureID']]
measure_dict = dict(zip(measure_df['MeasureID'], zip(measure_df['Short_Question_Text'], measure_df['Measure'])))
category_df = df_all[['Category','CategoryID']]
category_dict = dict(zip(category_df['Category'],category_df['CategoryID']))

In [ ]:
# Drop rows with US as state
df_all = df_all.loc[df_all['StateAbbr'] != 'US']

# Drop other relevant columns
df_all.drop(columns=['DataSource','Data_Value_Unit','CategoryID','DataValueTypeID',
                     'StateDesc', 'MeasureID','Year','Measure'],inplace=True)
df_all.columns

Let's quickly check for duplicates since we had to download all of the PLACES datasets separately to accomodate the year range in the SDOH dataset.

In [ ]:
dups = df_all.duplicated()
dups.value_counts()

In [ ]:
# Drop identified duplicates
df_all.drop_duplicates(ignore_index=True, inplace=True)
df_all.info()

### Statistical Analysis

Before we move onto defining the target, we should look at the numerical distribution of the `Data_Value` and `TotalPopulation` columns. 

We can see there are some outliers in both columns we should consider visualizing and potentially dropping to get a more accurate representation of our distributions.

We will want to consider normalizing the `Data_Value` column since the values are represented differently based on type. We can also consider weighing it if we consider one type more important than another.

In [ ]:
df_all[['Data_Value','TotalPopulation']].describe()

When visualizing these features, we can estimate that any `Data_Value` above ~85 is an outlier. It is harder to determine for `TotalPopulation`. We should look at the values of these more closely.

In [ ]:
fig, ax = plt.subplots(ncols=2,figsize=(12,6))
sns.boxplot(df_all['Data_Value'],ax=ax[0])
ax[0].set_title('Data Value Distribution')

sns.boxplot(df_all['TotalPopulation'],ax=ax[1])
ax[1].set_title('Total Population Distribution');

Taking closer at the number of rows that are outside of the the IQR, there are quite a few rows with population and data value outside of bounds.

In [ ]:
for col in df_all[['TotalPopulation', 'Data_Value']].columns:
    Q1 = df_all[col].quantile(.25)
    Q3 = df_all[col].quantile(.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df_all[(df_all[col] < lower_bound) | (df_all[col] > upper_bound)]
    print(f'{col} outliers:',outliers.shape[0])

Double checking each data type to see if they behave differently in relation to it's own data type vs collectively we see there is indeed a difference.

In [ ]:
total_outliers = 0
outlier_info = []
for data, data_type in [('Data_Value', 'Data_Value_Type')]:
    # IQR for 'Data_Value' filtered by 'Data_Value_Type'
    grouped = df_all.groupby(data_type)
    for name, group in grouped:
        Q1 = group[data].quantile(0.25)
        Q3 = group[data].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Detecting outliers
        outliers = group[(group[data] < lower_bound) | (group[data] > upper_bound)]
        num_outliers = outliers.shape[0]
        total_outliers += num_outliers
        outlier_info.append((name, outliers.shape[0]))

# Print outlier counts for each type
for name, count in outlier_info:
    print(f'{name} outliers: {count}')

print(f'Total outliers: {total_outliers}')

Doing the same with total population, we don't see the same difference so we can treat `TotalPopulation` collectively in the dataset.

In [ ]:
total_outliers = 0
outlier_info = []
for data, data_type in [('TotalPopulation', 'Data_Value_Type')]:
    # IQR for 'Data_Value' filtered by 'Data_Value_Type'
    grouped = df_all.groupby(data_type)
    for name, group in grouped:
        Q1 = group[data].quantile(0.25)
        Q3 = group[data].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Detecting outliers
        outliers = group[(group[data] < lower_bound) | (group[data] > upper_bound)]
        num_outliers = outliers.shape[0]
        total_outliers += num_outliers
        outlier_info.append((name, outliers.shape[0]))

# Print outlier counts for each type
for name, count in outlier_info:
    print(f'{name} outliers: {count}')

print(f'Total outliers: {total_outliers}')

In [ ]:
# Set to store indices of outliers
outlier_indices = set()  # Set to store indices of outliers

# Identify 'TotalPopulation' outliers
col = 'TotalPopulation'
Q1 = df_all[col].quantile(0.25)
Q3 = df_all[col].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = df_all[(df_all[col] < lower_bound) | (df_all[col] > upper_bound)]
outlier_indices.update(outliers.index)


col = 'Data_Value'
data_type_col = 'Data_Value_Type'
grouped = df_all.groupby(data_type_col)
for name, group in grouped:
    Q1 = group[col].quantile(0.25)
    Q3 = group[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Detecting outliers within each group
    outliers = group[(group[col] < lower_bound) | (group[col] > upper_bound)]
    outlier_indices.update(outliers.index)

df_norm = df_all.drop(index=outlier_indices).copy()


print(df_all.shape)
print(df_norm.shape)

It looks like our Data Value distribution for each Data Value Type isn't normal. So before we move on using it to create the HDI we want to create a new column with the normalize values using the `RobustScaler` since we have been using IQR to identify outliers.

In [ ]:
fig, ax = plt.subplots(ncols=3,figsize=(15,8))

sns.kdeplot(df_norm['Data_Value'].loc[df_norm['Data_Value_Type']=='crude prevalence'],ax=ax[0])
ax[0].set_title('Crude Prevalance')
sns.kdeplot(df_norm['Data_Value'].loc[df_norm['Data_Value_Type']=='age-adjusted prevalence'],ax=ax[1])
ax[1].set_title('Age-adjusted Prevalance')
sns.kdeplot(df_norm['Data_Value'].loc[df_norm['Data_Value_Type']=='percentage'],ax=ax[2])
ax[2].set_title('Percentage')
plt.tight_layout();

In [ ]:
# Initialize RobustScaler
scaler = RobustScaler()

# Create an empty column for scaled values in the original DataFrame
df_norm.loc[:, 'Scaled_Value'] = pd.NA

# Process each data type separately
for data_type in df_norm['Data_Value_Type'].unique():
    # Filter the df to only include rows of the current data type
    subset = df_norm[df_norm['Data_Value_Type'] == data_type]
    
    # Reshape is used because scaler expects a 2D array
    scaled_values = scaler.fit_transform(subset['Data_Value'].values.reshape(-1, 1))

    # Assign the scaled values back to the appropriate rows in the main DataFrame
    df_norm.loc[subset.index, 'Scaled_Value'] = scaled_values.flatten()

# Ensure the scaled value is a float
df_norm['Scaled_Value'] = df_norm['Scaled_Value'].astype('float')


df_norm.info()

### Defining Health Disparity Index

Now that our dataframe is aligned with our project goals, we can define our target variable `Health_Disparity_Index`. 

To start, we will create a general index taking the sum of all scaled values for each location and create a feature in the full dataset to represent this disparity index. 

In [ ]:
# Aggregate Scaled_Value per Geoplocation
sum_idx = df_norm.groupby('Geolocation')['Scaled_Value'].sum().reset_index()
sum_idx.rename({'Scaled_Value': 'Sum_Idx'},axis=1,inplace=True)

# Copy dataframe
df_sum_idx = df_norm.copy()

# Create new column
df_sum_idx = df_sum_idx.merge(sum_idx,on='Geolocation')
df_sum_idx.info()

Comparing random samples of `Sum_Idx` vs `Data_Value` shows us that there is a lot more uniformity in the `Scaled_Value` before aggregation of all of the data value types. This makes sense since our aggregated `Sum_Idx` is combining all values per geolocation, creating and therefore creates less variability when visualizing by location.

In [ ]:
# Helper plotting function

def idx_plot(data,y1,y2,y3,y1_name,y2_name,y3_name):
    fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(15, 12))

    # Visualize by State
    # Plot by y1
    sns.barplot(data=data, hue='StateAbbr', y=y1.sample(n=100000), 
                palette='Set2', ax=ax[0,0],legend=False)
    ax[0,0].set_title(f'{y1_name} by State')
    ax[0,0].set_xlabel('State')
    
    # Plot by y2
    sns.barplot(data=data, hue='StateAbbr', y=y2.sample(n=100000), 
                palette='Set2', ax=ax[0,1],legend=False)
    ax[0,1].set_title(f'{y2_name} by State')
    ax[0,1].set_xlabel('State') 

    # Plot by y3
    sns.barplot(data=data, hue='StateAbbr', y=y3.sample(n=100000), 
                palette='Set2', ax=ax[0,2],legend=False)
    ax[0,2].set_title(f'{y3_name} by State')
    ax[0,2].set_xlabel('State') 
    
    # Visualize by Geolocation
    # Plot by y1
    sns.scatterplot(data=data.sample(n=1000), x='Geolocation', y=y1, ax=ax[1, 0])
    ax[1, 0].set_title(f'{y1_name} by Geolocation')
    ax[1, 0].set_xlabel('Geolocation')

    ax[1, 0].set_xticks([])
    
    # Plot by y2
    sns.scatterplot(data=data.sample(n=1000), x='Geolocation', y=y2, ax=ax[1, 1])
    ax[1, 1].set_title(f'{y2_name} by Geolocation')
    ax[1, 1].set_xlabel('Geolocation')

    ax[1, 1].set_xticks([])

    # Plot by y3
    sns.scatterplot(data=data.sample(n=1000), x='Geolocation', y=y3, ax=ax[1, 2])
    ax[1, 2].set_title(f'{y3_name} by Geolocation')
    ax[1, 2].set_xlabel('Geolocation')

    ax[1, 2].set_xticks([])

    plt.tight_layout() 
    plt.show();

idx_plot(df_sum_idx,df_sum_idx['Sum_Idx'],df_sum_idx['Scaled_Value'],df_sum_idx['Data_Value'],'Sum Idx','Scaled Value','Data Value')

Let's see what our distribution looks like with an 'Avg_Idx' vs using raw sum of scaled values. 

In [ ]:
# Aggregate Scaled_Value per Geoplocation
avg_idx = df_norm.groupby('Geolocation')['Scaled_Value'].mean().reset_index()
avg_idx.rename({'Scaled_Value': 'Avg_Idx'},axis=1,inplace=True)

# Copy dataframe
df_avg_idx = df_norm.copy()

# Create new column
df_avg_idx = df_avg_idx.merge(avg_idx,on='Geolocation')
df_avg_idx.info()

When we create an `Avg_Idx` we now see our geolocation distribution but on a different scale and with more goeographical distinction. 

> This would make sense if for example, this segregation is representing rural vs urban areas.

In [ ]:
idx_plot(df_avg_idx,df_avg_idx['Avg_Idx'],df_avg_idx['Scaled_Value'],df_avg_idx['Data_Value'],'Avg Idx','Scaled Value','Data Value')

Lastly we will add a population weight to each scaled value, this will adjust our results to consider the density of a region when calculating the HDI.

In [ ]:
# Aggregate Scaled_Value per Geoplocation
pw_idx = df_norm.groupby('Geolocation')['TotalPopulation'].sum().reset_index()
pw_idx.rename({'TotalPopulation': 'GeoPop'},axis=1,inplace=True)

# Copy dataframe
df_pw_idx = df_norm.copy()

# Create new column
df_pw_idx = df_pw_idx.merge(pw_idx,on='Geolocation')

# Calculate weights
df_pw_idx['PopWeight'] = df_pw_idx['TotalPopulation']/df_pw_idx['GeoPop']

# Calculate weighted average of health measures
weighted_avg = df_pw_idx.groupby('Geolocation').apply(lambda x: (x['Scaled_Value']*x['PopWeight']).sum()).reset_index(name='Weighted_Idx')

df_pw_idx = df_pw_idx.merge(weighted_avg,on='Geolocation')

df_pw_idx.info()

Looks like our weighted average provides more of a spread than the `Avg_Idx, and the same relationship as the `Avg_Idx` which means population definitely has an impact on the HDI and geolocation does as well.

In [ ]:
idx_plot(df_pw_idx,df_pw_idx['Weighted_Idx'],df_pw_idx['Scaled_Value'],df_pw_idx['Data_Value'],'Weighted Idx','Scaled Value','Data Value')

### Geolocation to HDI Visualization

Before we finalize our decision to use the population weighted HDI, let's visualize the index in comparison to population using the `Geolocation`. To do so, we first have to convert the feature to wkt (well-known text) format for use with the geopandas library.

In [ ]:
df_pw_idx.loc[:,'Geolocation']= df_pw_idx['Geolocation'].apply(lambda x: x.upper())
df_pw_idx.loc[:,'Geolocation'] = df_pw_idx['Geolocation'].apply(wkt.loads)
df_pw_idx.info()

Now we can create a GeoDataFrame with `Geolocation` as the geometrical feature.

In [ ]:
gdf_pw = gpd.GeoDataFrame(df_pw_idx,geometry='Geolocation')

Looking at population and the HDI across all of the geolocations we can see that the higher the population, the lower the HDI. In particular we can see that in part of the west coast, Florida, and part of the east coast demonstrate this inverse relationship.

> This inverse relationship may make sense when you consider factors such as but not limited to; greater access to care, diversity of services, and public health funding in areas with more population density.

This further confirms our decision to use population as a weight for our HDI definition.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15, 6))

# Plotting 'TotalPopulation'
gdf_pw.plot(column='TotalPopulation', legend=True, ax=ax[0])
ax[0].set_title('Total Population')
ax[0].set_xlim(-130, -65)
ax[0].set_ylim(25, 50)

# Plotting 'Sum_Idx'
gdf_pw.plot(column='Weighted_Idx', legend=True, ax=ax[1])
ax[1].set_title('Weighted HDI')
ax[1].set_xlim(-130, -65)
ax[1].set_ylim(25, 50)

plt.tight_layout();

## 3. Data Preparation

Since our dataset is so large we will train our model on a smaller training set to save computational power and time. We are also going to drop the columns we used to create our `Weighted_Idx`. 

> In future itterations we would want to consider using `Geolocation` to create location features and explore spatial relationships. However, to keep computational cost relatively low, we will note this as a Phase 2 addition.

We are using `StratifiedShuffleSplit` to create subsets of our data while maintaining proportions of the target variable. Running a subset of the data will give us a way to understand model performance while maintaining computational efficiency. Once we have found the best fitting model, we will run it on our extra hold out data we didn't train on, further validating our results.

In [ ]:
# Define X, y, and split data

X = df_pw_idx.drop(columns=['Weighted_Idx','GeoPop','PopWeight','Scaled_Value','Geolocation'])
y = df_pw_idx['Weighted_Idx']

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=.2)

# Initialize StratifiedShuffleSplit
strat_splitter = StratifiedShuffleSplit(n_splits=1,train_size=.3)

# Generate indices for the stratified sample
for train_index, _ in strat_splitter.split(X_train, y_train):
    X_sampled = X.iloc[train_index]
    y_sampled = y.iloc[train_index]
    
# Generate hold-out sets for final model evaluation
mask = np.zeros(len(X_train), dtype=bool)
mask[train_index] = True
X_holdout = X_train.iloc[~mask]
y_holdout = y_train.iloc[~mask]

# Conduct new train test split for our subset we will using be training
X_samp_train, X_samp_test, y_samp_train, y_samp_test = train_test_split(
    X_sampled, y_sampled, stratify=y_sampled, test_size=0.2)
X.info()

### Base Model and Metrics

`StandardScaler` is used on all our numerical data to make sure we are comparing our features on the same scale. And `OneHotEncoder` is used on the rest of the object or categorical categories. 

Our base model will be a standard `LinearRegression`.

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import PolynomialFeatures

# Define num and cat subpipes
subpipe_num = Pipeline(steps=[
    ('ss', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2))])
subpipe_cat = Pipeline(steps=[('ohe', OneHotEncoder(sparse_output=False,handle_unknown='ignore'))])

# Initiate ColumnTranformer
CT = ColumnTransformer(transformers=[('subpipe_num',subpipe_num,[4,5]),
                                     ('subpipe_cat',subpipe_cat,[0,1,2,3,6,7])],
                       remainder='passthrough')

# Initial Pipeline
init_pipe = Pipeline(steps=[('ct',CT),
                           ('model',DummyRegressor())])

In [ ]:
init_pipe.fit(X_samp_train,y_samp_train)

It looks like our primary RMSE metric is pretty good. However, it is expected that it would be small considering the range of our HDI so we will aim to improve these further. $R^2$ can also stand to improve. Thankfully, we don't see much overfitting or underfitting.

We will keep these metrics in mind as we test other models.

In [ ]:
# Baseline Metrics
print('Train RSME:', root_mean_squared_error(y_samp_train,init_pipe.predict(X_samp_train)))
print('Test RSME:', root_mean_squared_error(y_samp_test,init_pipe.predict(X_samp_test)))
print('Train R-Squared:', r2_score(y_samp_train,init_pipe.predict(X_samp_train)))
print('Test R-Squared:', r2_score(y_samp_test,init_pipe.predict(X_samp_test)))
print('Train MAE:', mean_absolute_error(y_samp_train,init_pipe.predict(X_samp_train)))
print('Test MAE:', mean_absolute_error(y_samp_test,init_pipe.predict(X_samp_test)))

Even after dropping a significant amount of features, we are left with fairly high dimentionality. This is something we will want to consider as we test and refine our models.

In [ ]:
X_train_trans = init_pipe['ct'].transform(X_samp_train)
df_trans = pd.DataFrame(X_train_trans,columns=init_pipe['ct'].get_feature_names_out())
print(df_trans.shape)
df_trans

## 4. Linear Regression Model

We will start by testing different regularization techniques to address our moderate $R^2$ and improve our error metrics in this high dimensional space.

In [ ]:
# Helper function to create new pipeline

def new_pipe(estimator):
    return Pipeline(steps=[
        ('ct', CT),  
        ('model', estimator)
    ])

## SVR

In [2]:
from sklearn.svm import SVR

# Replace in pipeline

svr_pipe = new_pipe(SVR())
svr_pipe.fit(X_samp_train,y_samp_train)

NameError: name 'new_pipe' is not defined

In [ ]:
mcv_svr = MetricsCV(svr_pipe,'svr',X_samp_train,y_samp_train)

params_svr = {'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'model__C': [0.1, 1, 10, 100, 1000],
              'model__degree': [2,3,4,5],
              'model__gamma': [0.01, 0.1, 1, 10]
}

mcv_svr.run_rcv(params=params_svr,scoring="neg_root_mean_squared_error",
                                             verbose=3,n_iter=5)

In [ ]:
# Get validation scores and best estimator
val_scores, svr_best = mcv_svr.rcv_metrics()

In [ ]:
test_scores = mcv_svr.test_metrics(X_samp_test,y_samp_test)
display(val_scores)
test_scores

### Lasso

In [43]:


lasso_pipe = new_pipe(Lasso(max_iter=1000))
lasso_pipe.fit(X_samp_train,y_samp_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('subpipe_num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  [4, 5]),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  [0, 1, 2, 3, 6, 7])])),
                ('model', Lasso())])

In [99]:
mcv_lasso = MetricsCV(lasso_pipe,'Lasso',X_samp_train,y_samp_train)

space_lasso = [Real(10**-4,10**3,'log-uniform',name='alpha'),
              Categorical(['cyclic','random'],name='selection')]



val_scores, lasso_best = mcv_lasso.objective(space=space_lasso,scoring='neg_root_mean_squared_error',
                                           verbose=3, n_calls=15, n_initial_points=5,n_jobs=-1,n_points=4,dx_stopper=1e-4)

Iteration No: 1 started. Evaluating function at random point.


python(10188) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(10189) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(10190) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(10191) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(10192) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(10193) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(10194) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(10195) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 50.7560
Function value obtained: 0.0643
Current minimum: 0.0643
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 49.1500
Function value obtained: 0.0643
Current minimum: 0.0643
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 45.3577
Function value obtained: 0.0643
Current minimum: 0.0643
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 47.9286
Function value obtained: 0.0643
Current minimum: 0.0643
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 53.5997
Function value obtained: 0.0643
Current minimum: 0.0643
Iteration No: 6 started. Searching for the next optimal point.
Iteration No: 6 ended. Search

In [100]:
# View best params
lasso_best['model'].get_params

<bound method BaseEstimator.get_params of Lasso(alpha=0.0001, selection='random')>

Looks like our Lasso model did slightly worse in error metrics and significantly worse in $R^2$. 

In [101]:
test_scores = mcv_lasso.test_metrics(X_samp_test,y_samp_test)
display(val_scores)
test_scores

/Users/elinarankova/anaconda3/envs/learn-env-lite-2/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/elinarankova/anaconda3/envs/learn-env-lite-2/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


,Model Name,Val Train Score,Val Test Score
0,Lasso,0.047102,0.047102


,Model Name,Train Score,Test Score,Train R2,Test R2,Train MAE,Test MAE
0,Lasso,0.047084,0.047077,0.46457,0.464606,0.039646,0.039654


### Ridge

Now we can assess the Ridge model in comparison to Lasso to see which type of regularization is best for our problem.

In [47]:
# Replace in pipeline

ridge_pipe = new_pipe(Ridge(random_state=seed_value))
ridge_pipe.fit(X_samp_train,y_samp_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('subpipe_num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  [4, 5]),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  [0, 1, 2, 3, 6, 7])])),
                ('model', Ridge(random_state=42))])

In [104]:
mcv_ridge = MetricsCV(ridge_pipe,'Ridge',X_samp_train,y_samp_train)

params_ridge = {'model__alpha': [.0001,.001,.01,100,1000],
                'model__solver': ['auto','cholesky','lsqr','sag'],
                'model__tol': [.0001,.001,.01]
}

mcv_ridge.run_rcv(params=params_ridge,scoring="neg_root_mean_squared_error",
                                             verbose=3,n_iter=5)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END model__alpha=100, model__solver=cholesky, model__tol=0.0001;, score=(train=-0.041, test=-0.041) total time=  22.5s
[CV 2/5] END model__alpha=100, model__solver=cholesky, model__tol=0.0001;, score=(train=-0.041, test=-0.041) total time=  20.7s
[CV 3/5] END model__alpha=100, model__solver=cholesky, model__tol=0.0001;, score=(train=-0.041, test=-0.041) total time=  20.8s
[CV 4/5] END model__alpha=100, model__solver=cholesky, model__tol=0.0001;, score=(train=-0.041, test=-0.041) total time=  21.3s
[CV 5/5] END model__alpha=100, model__solver=cholesky, model__tol=0.0001;, score=(train=-0.041, test=-0.041) total time=  21.4s
[CV 1/5] END model__alpha=100, model__solver=sag, model__tol=0.01;, score=(train=-0.041, test=-0.041) total time=  16.5s
[CV 2/5] END model__alpha=100, model__solver=sag, model__tol=0.01;, score=(train=-0.041, test=-0.041) total time=  19.0s
[CV 3/5] END model__alpha=100, model__solver=sag, model__t

{'model__tol': 0.01, 'model__solver': 'cholesky', 'model__alpha': 0.0001}

In [105]:
# Get validation scores and best estimator
val_scores, ridge_best = mcv_ridge.rcv_metrics(val_df=val_scores)

In [106]:
# View best params
ridge_best['model'].get_params

<bound method BaseEstimator.get_params of Ridge(alpha=0.0001, random_state=42, solver='cholesky', tol=0.01)>

Our Ridge model definitely helped lower our error metrics as well as raise $R^2$ vs our Lasso model. However, These scores are about as good as our base metrics.

In [123]:
test_scores = mcv_ridge.test_metrics(X_samp_test,y_samp_test,test_df=test_scores)
display(val_scores)
test_scores

/Users/elinarankova/anaconda3/envs/learn-env-lite-2/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/elinarankova/anaconda3/envs/learn-env-lite-2/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


,Model Name,Val Train Score,Val Test Score
0,Lasso,0.047102,0.047102
1,Ridge,0.040319,0.040878


,Model Name,Train Score,Test Score,Train R2,Test R2,Train MAE,Test MAE
1,Ridge,0.036465,0.036466,0.678844,0.678765,0.032773,0.032738
0,Lasso,0.047084,0.047077,0.464570,0.464606,0.039646,0.039654


## Neural Network

Since we are not seeing a big enough improvement after testing both Ridge and Lasso regularization, to address the low $R^2$ and lack of representation of the variance in our data we will work on a neural network using the `keras` within the `tensorflow` library.

> Due to this environment's incompatibility with the `scikeras`library from which we would download the `KerasRegression` wrapper to insert our neural network into the pipeline, a custom class was created to mimic the function of the wrapper.

In [55]:
import inspect
class KerasRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, build_fn=None, random_state=None, **kwargs):
        self.build_fn = build_fn
        self.random_state = random_state
        self.kwargs = kwargs  # Store constructor kwargs
        self.model_ = None
        self.history_ = None  # To store the fit history

    def fit(self, X, y, **fit_params):

        # Prepare model_kwargs based on the signature of build_fn
        # It filters out fit_params and includes only relevant kwargs for build_fn
        sig_params = inspect.signature(self.build_fn).parameters
        build_kwargs = {k: v for k, v in self.kwargs.items() if k in sig_params}
        build_kwargs.update(fit_params.get('build_kwargs', {}))  # Include any build-specific parameters from fit_params
        
        # Assuming 'input_dim' must be specified if not already included
        if 'input_dim' not in build_kwargs and 'input_shape' not in build_kwargs and X is not None:
            build_kwargs['input_dim'] = X.shape[1]

        self.model_ = self.build_fn(random_state=self.random_state, **build_kwargs)
        self.history_ = self.model_.fit(X, y, **{k: v for k, v in fit_params.items() if k not in ['build_kwargs']})
        return self

    def predict(self, X):
        return self.model_.predict(X).flatten()

    def score(self, X, y):
        predictions = self.predict(X)
        mse = mean_squared_error(y, predictions)
        return -mse  

    def get_params(self, deep=True):
        return {'build_fn': self.build_fn, 'random_state': self.random_state, **self.kwargs}

    def set_params(self, **params):
        # Update kwargs with any set parameters
        self.kwargs.update(params)
        # Ensure that all parameters are set correctly
        for key, value in params.items():
            setattr(self, key, value)
        return self


Here we will play with the neural network architecture until we get scores providing us with progress in our $R^2$ and error metrics.

In [56]:
# Define a function to create the Keras model
from tensorflow.keras.layers import LeakyReLU
def build_model(optimizer='adam', 
                units=100, 
                activation='relu',
                input_dim=4425, 
                random_state=seed_value):
    
    if random_state is not None:
        np.random.seed(random_state)
        tf.random.set_seed(random_state)
        
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(units, activation=activation),
        Dense(units, activation=activation),
        Dense(units, activation=activation),
        Dense(units, activation=activation),
        Dense(units, activation=activation),
        Dense(units, activation=activation),
        Dense(1000, activation=activation),
        Dropout(0.5),
        Dense(units, activation=activation),
        Dense(units, activation=activation, kernel_regularizer=l1_l2(l1=.01,l2=.01)),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

Since we have limited CPU power and want to get efficient yet accurate results, we will implement early stopping as well as reduce our learning rate as the difference in our loss function reaches a minimum.

In [57]:
# Wrap the model with KerasRegressor
nn_model = KerasRegressor(build_fn=build_model,random_state=seed_value)

# Define baseline error to achieve based off baseline hold out RMSE score
baseline_mse = 0.03646619211292927**2 # Calculating the MSE from RMSE

# Define early stopping parameters
early_stopping = EarlyStopping(monitor='val_loss',min_delta=1e-6, patience=30, verbose=3, 
                               mode='min', restore_best_weights=True, baseline=baseline_mse)

# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=30, min_lr=0.0001, verbose=3)

fit_params = {
    'model__epochs': 200,
    'model__batch_size': 1000,
    'model__verbose': 1,
    'model__callbacks': [early_stopping, reduce_lr],
    'model__validation_split': 0.2
}

# Replace in pipeline
nn_pipe = new_pipe(KerasRegressor(build_model))

In [58]:
# Fit model to training data
nn_pipe.fit(X_samp_train,y_samp_train, **fit_params)

Epoch 1/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 6.9820 - mean_squared_error: 0.0124 - val_loss: 1.4613 - val_mean_squared_error: 0.0015 - learning_rate: 0.0010
Epoch 2/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.7661 - mean_squared_error: 0.0018 - val_loss: 0.0171 - val_mean_squared_error: 0.0018 - learning_rate: 0.0010
Epoch 3/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0152 - mean_squared_error: 0.0017 - val_loss: 0.0137 - val_mean_squared_error: 0.0015 - learning_rate: 0.0010
Epoch 4/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0136 - mean_squared_error: 0.0015 - val_loss: 0.0137 - val_mean_squared_error: 0.0015 - learning_rate: 0.0010
Epoch 5/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0135 - mean_squared_error: 0.0015 - val_loss: 0.0133 - val_mean_squared_error: 0.0015 - learning_rate: 0.0010
Epoch 6/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0132 - mean_squared_error: 0.0014 - val_loss: 0.0133 - val_mean_sq

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('subpipe_num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  [4, 5]),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  [0, 1, 2, 3, 6, 7])])),
                ('model',
                 KerasRegressor(build_fn=<function build_model at 0x3af0d6ca0>))])

Comparing our results not unexpectedly, the Neural Network is overfitting but the train scores are improving!

In [124]:
mcv_nn = MetricsCV(nn_pipe,'Neural Network',X_samp_train,y_samp_train)

# Get validation and test metrics
val_scores = mcv_nn.cross_val(scoring='neg_root_mean_squared_error',val_df=val_scores,verbose=3)
test_scores = mcv_nn.test_metrics(X_samp_test,y_samp_test,test_df=test_scores,test_pipe=nn_pipe)

display(val_scores)
test_scores

3329/3329 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.8175 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 2s 491us/step
[CV] END ................., score=(train=-0.038, test=-0.040) total time=  11.0s
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.8207 - mean_squared_error: 0.0030
833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 2s 517us/step
[CV] END ................., score=(train=-0.039, test=-0.040) total time=  12.3s
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.8147 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 2s 450us/step
[CV] END ................., score=(train=-0.039, test=-0.040) total time=  11.0s
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.8303 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 2s 486us/step
[CV] END ................., score=(train=-0.040, test=-0

/Users/elinarankova/anaconda3/envs/learn-env-lite-2/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


1041/1041 ━━━━━━━━━━━━━━━━━━━━ 1s 529us/step


/Users/elinarankova/anaconda3/envs/learn-env-lite-2/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


4161/4161 ━━━━━━━━━━━━━━━━━━━━ 2s 528us/step
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 1s 511us/step
4161/4161 ━━━━━━━━━━━━━━━━━━━━ 2s 455us/step
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step


,Model Name,Val Train Score,Val Test Score
0,Lasso,0.047102,0.047102
1,Ridge,0.040319,0.040878
2,Neural Network,0.039151,0.040340


,Model Name,Train Score,Test Score,Train R2,Test R2,Train MAE,Test MAE
0,Ridge,0.036465,0.036466,0.678844,0.678765,0.032773,0.032738
2,Neural Network,0.031931,0.041014,0.753744,0.593644,0.025768,0.034124
1,Lasso,0.047084,0.047077,0.464570,0.464606,0.039646,0.039654


## VotingRegressor

Let's see if we can get the best of the strongest models so far to get a holistic improvement in metrics. Hopefully this addresses the overfitting seen in the Neural Network

In [125]:
# Create weighted averaging 

estimators = [
    ('ridge',ridge_best['model']),
    ('nn',nn_pipe['model'])]

voting_pipe = new_pipe(VotingRegressor(estimators=estimators))

voting_pipe.fit(X_samp_train,y_samp_train)

4161/4161 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.6862 - mean_squared_error: 0.0026


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('subpipe_num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  [4, 5]),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  [0, 1, 2, 3, 6, 7])])),
                ('model',
                 VotingRegressor(estimators=[('ridge',
                                              Ridge(alpha=0.0001,
                                                    random_state=42,
                                                    solver='cholesky',
                                                    tol=0.01)),
                                             ('nn',
                                              KerasRegressor(build_fn=<function build_model at 0x3af0d6ca0>))]))])

In [126]:
mcv_voting = MetricsCV(voting_pipe,'Voting',X_samp_train,y_samp_train)

params_voting = {"model__weights": [[0.57, 0.43],[0.55, 0.45]]}



mcv_voting.run_rcv(params=params_voting,scoring="neg_root_mean_squared_error",
                                             verbose=3,n_iter=2)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.8149 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 2s 454us/step
[CV 1/5] END model__weights=[0.57, 0.43];, score=(train=-0.036, test=-0.038) total time=  27.8s
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 0.8170 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 1s 440us/step
[CV 2/5] END model__weights=[0.57, 0.43];, score=(train=-0.037, test=-0.038) total time=  30.7s
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.8252 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 1s 581us/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 2s 644us/step
[CV 3/5] END model__weights=[0.57, 0.43];, score=(train=-0.036, test=-0.037) total time=  32.1s
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.8387 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 0s 546us

{'model__weights': [0.57, 0.43]}

In [127]:
# Get validation scores and best estimator
val_scores, voting_best = mcv_voting.rcv_metrics(val_df=val_scores)

In [128]:
# View best params
voting_best['model'].get_params

<bound method _BaseHeterogeneousEnsemble.get_params of VotingRegressor(estimators=[('ridge',
                             Ridge(alpha=0.0001, random_state=42,
                                   solver='cholesky', tol=0.01)),
                            ('nn',
                             KerasRegressor(build_fn=<function build_model at 0x3af0d6ca0>))],
                weights=[0.57, 0.43])>

In [129]:
test_scores = mcv_voting.test_metrics(X_samp_test,y_samp_test,test_df=test_scores)
display(val_scores)
test_scores

4161/4161 ━━━━━━━━━━━━━━━━━━━━ 2s 497us/step


/Users/elinarankova/anaconda3/envs/learn-env-lite-2/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


1041/1041 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step


/Users/elinarankova/anaconda3/envs/learn-env-lite-2/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


4161/4161 ━━━━━━━━━━━━━━━━━━━━ 2s 536us/step
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step
4161/4161 ━━━━━━━━━━━━━━━━━━━━ 2s 515us/step
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step


,Model Name,Val Train Score,Val Test Score
0,Lasso,0.047102,0.047102
1,Ridge,0.040319,0.040878
2,Neural Network,0.039151,0.040340
3,Voting,0.036623,0.037665


,Model Name,Train Score,Test Score,Train R2,Test R2,Train MAE,Test MAE
0,Ridge,0.036465,0.036466,0.678844,0.678765,0.032773,0.032738
3,Voting,0.036516,0.036717,0.677949,0.674325,0.032619,0.032779
1,Neural Network,0.031931,0.041014,0.753744,0.593644,0.025768,0.034124
2,Lasso,0.047084,0.047077,0.464570,0.464606,0.039646,0.039654


## StackingRegressor

Lastly, we will try another ensemble model with the `StackingRegressor`. We are going to include the `VotingRegressor` to see if we can capture more of the patterns through stacking.

The idea that the `StackingRegressor` will be able to learn the complex patterns not being captured by voting alone. Since each model in the ensemble might be picking up different patterns, evaluating each separately and learning from the different errors creates a robust new estimator that can address the variety in the data we haven't been able to capture.

In [130]:
estimators = [
    ('ridge',ridge_best['model']),
    ('nn',nn_pipe['model']),
    ('voting',voting_pipe)]

stacking_pipe = new_pipe(StackingRegressor(estimators=estimators))

stacking_pipe.fit(X_samp_train,y_samp_train)

4161/4161 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.6993 - mean_squared_error: 0.0027
4161/4161 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step - loss: 0.6827 - mean_squared_error: 0.0021
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.8297 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 1s 756us/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.8205 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.8147 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - loss: 0.8108 - mean_squared_error: 0.0029
833/833 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 39s 11ms/step - loss: 0.8297 - mean_squared_error: 0.0030
833/833 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 60s 17ms/step - loss: 0.8297 - mean_squared_error: 0.0023
833/833 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 24s 7ms/step 

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('subpipe_num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  [4, 5]),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  [0, 1, 2, 3, 6, 7])])),
                ('model',
                 StackingRegressor(estimators=[('ridge',
                                                Ridge(alpha=0.0001,
                                                      random_state=42,
                                                      solver=...
                                                                                                  Pipeline(steps=[('ss',
                                                                                                                   StandardScaler())]),
                                                                                                  [4,
                                                                                                   5]),
                                                                                                 ('subpipe_cat',
                                                                                                  Pipeline(steps=[('ohe',
                                                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                                                 sparse_output=False))]),
                                                                                                  [0,
                                                                                                   1,
                                                                                                   2,
                                                                                                   3,
                                                                                                   6,
                                                                                                   7])])),
                                                                ('model',
                                                                 VotingRegressor(estimators=[('ridge',
                                                                                              Ridge(alpha=0.0001,
                                                                                                    random_state=42,
                                                                                                    solver='cholesky',
                                                                                                    tol=0.01)),
                                                                                             ('nn',
                                                                                              KerasRegressor(build_fn=<function build_model at 0x3af0d6ca0>))]))]))]))])

In [134]:
mcv_stacking = MetricsCV(stacking_pipe,'Stacking',X_samp_train,y_samp_train)

# Get validation and test metrics
val_scores = mcv_stacking.cross_val(scoring='neg_root_mean_squared_error',val_df=val_scores,verbose=3)
test_scores = mcv_stacking.test_metrics(stacking_pipe,X_samp_test,y_samp_test,test_df=test_scores)

display(val_scores)
test_scores

3329/3329 ━━━━━━━━━━━━━━━━━━━━ 34s 9ms/step - loss: 0.8230 - mean_squared_error: 0.0029
3329/3329 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.8183 - mean_squared_error: 0.0023
2663/2663 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.9854 - mean_squared_error: 0.0030
666/666 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step
2663/2663 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.9659 - mean_squared_error: 0.0032
666/666 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step
2663/2663 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.9790 - mean_squared_error: 0.0032
666/666 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step
2663/2663 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 0.9729 - mean_squared_error: 0.0031
666/666 ━━━━━━━━━━━━━━━━━━━━ 0s 614us/step
2663/2663 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.9749 - mean_squared_error: 0.0032
666/666 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step
2663/2663 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 0.9802 - mean_squared_error: 0.0027
666/666 ━━━━━━━━━━━━━━━━━━━━ 1s 793us/step
2663/2663 ━━━━━━━━━━━━━━━━━━━━ 37s 13ms/s

KeyboardInterrupt: 

In [ ]:
# test_scores = mcv_stacking.test_metrics(X_samp_test,y_samp_test,test_df=test_scores)
# display(val_scores)
# test_scores

In [ ]:
# # Cross validate to verify results

# cv_results = cross_validate(stacking_pipe,X_samp_train,y_samp_train, verbose=3,
#                             cv=5, scoring=('r2','neg_root_mean_squared_error','neg_mean_absolute_error'),
#                             return_train_score=True)

In [ ]:
# # Baseline Metrics
# print('Train RSME:', root_mean_squared_error(y_samp_train,stacking_pipe.predict(X_samp_train)))
# print('Test RSME:', root_mean_squared_error(y_samp_test,stacking_pipe.predict(X_samp_test)))
# print('Train R-Squared:', r2_score(y_samp_train,stacking_pipe.predict(X_samp_train)))
# print('Test R-Squared:', r2_score(y_samp_test,stacking_pipe.predict(X_samp_test)))
# print('Train MAE:', mean_absolute_error(y_samp_train,stacking_pipe.predict(X_samp_train)))
# print('Test MAE:', mean_absolute_error(y_samp_test,stacking_pipe.predict(X_samp_test)))

In [ ]:
# # CV Metrics
# print('Val Train RSME:', -cv_results['train_neg_root_mean_squared_error'].mean())
# print('Val Test RSME:', -cv_results['test_neg_root_mean_squared_error'].mean())
# print('Val Train R-Squared:', cv_results['train_r2'].mean())
# print('Val Test R-Squared:', cv_results['test_r2'].mean())
# print('Val Train MAE:', -cv_results['train_neg_mean_absolute_error'].mean())
# print('Val Test MAE:', -cv_results['test_neg_mean_absolute_error'].mean())

## Final Testing

To further solidify our findings, let's test the final model on our holdout data and test data set aside during data preparation.

In [ ]:
mcv_final = MetricsCV(stacking_pipe,"Stacking",X_holdout,y_holdout)

# Final holdout metrics
final_test_scores = mcv_final.test_metrics(X_holdout,y_holdout,test_df=final_test_scores)

In [ ]:
final_test_scores = mcv_final.test_metrics(X_test,y_test,test_df=final_test_scores)

In [ ]:
# import pickle

# # Attempt to serialize and deserialize the pipeline
# serialized_pipe = pickle.dumps(voting_pipe)
# deserialized_pipe = pickle.loads(serialized_pipe)

# # Test on sample data
# print(deserialized_pipe.predict(X_samp_train[:5]))


## Final Evaluation & Conclusion

[final evaluation]

**Recommendations**


**Data Limitation and Future Considerations:**

Our data set is quite large, while strategies were taken to compensate for the lack of machinery able to efficiently handle this amount of data, in future iterations it would be recommended to invest in more powerful processing to implement more advanced techniques such as `GridSearchCV` that may be more costly.

In addition, since we have `Geolocation` available we would want to create location features and explore spatial relationships through methods such as proximity clustering.

We also had to drop the year associeated with each record. In the future, we would either want to seek to add a year to SDOH measure records or conduct a sub analysis of the other categories provided in the PLACES datasets.

Lastly, there were incompatibilities with the `scikeras` package and the programming environment leading to a need for a custom KerasRegressor. We would want to resolve these issues and perform modeling within a compatible environment.